In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from src.utils import *
from src.graph2vec import ExtendedGraph2Vec, Ensemble_G2V

In [2]:
graphs, labels = read_stargazers_dataset("../datasets/github_stargazers")

order_dict = np.load("../datasets/github_stargazers/orderings.npy", allow_pickle=True)[
    ()
]

cv_fold = 64

tested_range = np.linspace(0, 1, 11)

WEIGHTING_FUNCTIONS = (
    {f"w_mean_{a:.2f}": ("w_mean", [a]) for a in tested_range}
    | {f"concat": ("concatenate", [])}
    | {f"p_proj_{a}": ("partial_projection", [0, a]) for a in (True, False)}
    | {f"w_proj_{a:.2f}": ("w_projection", [a]) for a in tested_range}
)

ens_model = Ensemble_G2V()

Loaded 12725 graphs


In [3]:
res = []
for measure, order in tqdm(order_dict.items()):
    ens_model.fit(graphs, orderings=order)
    for name, (func, args) in tqdm(WEIGHTING_FUNCTIONS.items(), leave=False):
        ens_model.set_weighting_function(func, *args)
        res.append(
            generate_results(
                ens_model.get_embedding(),
                labels=labels,
                classifier=LogisticRegression(max_iter=1000),
                method=f"{measure}_{name}",
                cv=cv_fold,
            )
        )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [7]:
pd.concat(res, ignore_index=True).to_csv("../datasets/github_stargazers/results.csv")